In [127]:
import os
import pickle
import faiss
import numpy as np
import re
from datetime import datetime
import requests
import json
import spacy

from zoneinfo import ZoneInfo

# Get system's local timezone
import tzlocal
local_tz = tzlocal.get_localzone()
# Make datetime.now() timezone-aware using local timezone
local_now = datetime.now(ZoneInfo(str(local_tz)))


print(local_tz)
print(local_now)


Asia/Kolkata
2025-06-18 02:22:38.502162+05:30


In [2]:
query = "Aditya with pratik in Goa or Nashik"

In [3]:
embed_dir = 'embed_store'

# loading up the indices from embed_dir
img_path_index = pickle.load(open(os.path.join(embed_dir, 'img_path_index.pkl'), 'rb'))
face_data = pickle.load(open(os.path.join(embed_dir, 'face_data.pkl'), 'rb'))
flatten_img_face_index = pickle.load(open(os.path.join(embed_dir, 'img_path_index_for_face.pkl'), 'rb'))
clip_embed = faiss.read_index(os.path.join(embed_dir, 'img_embeddings.bin'))
geo_data = np.load(os.path.join(embed_dir, 'geo_metadata.npy'), allow_pickle=True)

In [4]:
faces = set()
for face in face_data:
    if len(face['name'])>0:
        faces.add(face['name'].upper())
list(faces)

['YASH',
 'SHUBHAM',
 'ADITYA',
 'PRATIK',
 'AISHWARYA',
 'JINAY',
 'SHRUTI',
 'SAYALI',
 'NAMITA',
 'DHRUV']

In [5]:
for word in query.split(' '):
    if word.upper() in faces:
        print(word)


Aditya
pratik


In [6]:
def tokenize(query):
    return [tokens.strip().lower() for tokens in re.split(r",|:|'|\.|-|\s", query.split(sep="Location:")[-1]) if (tokens and len(tokens)>1)]

In [7]:
str(geo_data[0])


'Chungthang, Mangan, Sikkim, 737120, India'

In [8]:
geo_locs = set()
for address in geo_data:
    for word in tokenize(address):
        geo_locs.add(word.upper())
list(geo_locs)
print(len(geo_locs))

182


In [9]:
for word in query.split(' '):
    if word.upper() in faces:
        print(f'face: {word}')
    if word.upper() in geo_locs:
        print(f'location: {word}')


face: Aditya
face: pratik
location: Goa
location: Nashik


In [108]:
import spacy

nlp = spacy.load("en_core_web_lg")

def nlp_with_datetime_ner(query):
    doc = nlp(query)
    datetime_entities = [ent.text for ent in doc.ents if ent.label_ == "DATE" or ent.label_ == "TIME"]
    output_string = ", ".join(datetime_entities)
    return output_string


## DateTime Search

In [128]:
dt_data = np.load(os.path.join('.', 'datetime_metadata.npy'), allow_pickle=True)
dt_data = [str(date) for date in dt_data if date is not None]
print(f"Total datetimes loaded: {len(dt_data)}")

Total datetimes loaded: 10582


In [129]:
dt_data[:5]

['2023-03-18T19:44:56+05:30',
 '2016-07-20T15:27:22+05:30',
 '2022-10-19T08:41:55+05:30',
 '2021-03-30T09:14:05+05:30',
 '2021-03-30T12:12:50+05:30']

In [ ]:
# HANDLING DUCKLING INSIDE PYTHON:

import subprocess
import atexit

# Start Duckling
duckling_process = subprocess.Popen(
    ["stack", "exec", "duckling-example-exe"],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL
)

# Kill Duckling on exit
def cleanup_duckling():
    if duckling_process.poll() is None:  # Still running
        duckling_process.terminate()
        print("Duckling server terminated.")

atexit.register(cleanup_duckling)





def detect_grains(text):
    grains = set()
    if re.search(r'\b\d{1,2}\b', text):  # numbers like 27
        grains.add('day')
    if re.search(r'\b(jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec|march|april|june|july|august|september|october|november|december)\b', text, re.IGNORECASE):
        grains.add('month')
    if re.search(r'\b\d{4}\b', text):  # year like 2023
        grains.add('year')
    if re.search(r'\b\d{1,2}:\d{2}\b', text):  # time
        grains.add('minute')
        grains.add('hour')
    elif re.search(r'\b\d{1,2}\s*(am|pm)\b', text, re.IGNORECASE):
        grains.add('hour')
    # Add day and month grain if holidaybeta in duckling is found (add year on top if found using this function)
    return grains





'8010'

In [132]:
# "from this morning" doesn't work because it only give 'from' and not 'to', replace all "from" before parsing?
# "around sunset" doesn't work
# keep a sunset-sunrise etc paired with after, before, during, around etc for them and manually program 

queries = [
    "Images taken during Diwali 2023",
    "Photos from Monsoon 2024",
    "Pictures captured on Republic day",
    "Show me photos from Holi 2022",
    "Find images from September 2023",
    "Photos taken between October 15, 2024, and November 15, 2024",
    "Images from last year's Durga Puja",
    "Images of garba during navratri",
    "Pictures from this morning in Bengaluru",
    "Show me photos from December 2023 in Mumbai",
    "Images taken on a Sunday morning",
    "Photos captured on Independence Day 2021",
    "Pictures from the summer of 2020",
    "Show me images taken around sunset last Tuesday",
    "Photos from Ganesh Chaturthi 2025",
    "Images from March to May 2023 ",
    "Pictures taken during lunchtime on April 10, 2025",
    "Show me photos from last Christmas",
    "Find images from the New Year's Eve party 2023",
    "Photos from the past week in Delhi",
    "Images captured after 7 PM in Chennai",
    "Pictures taken on Ugadi 2024",
    "Show me photos from the winter of 2022",
    "Images from Vishu 2025",
    "Photos from around 6 AM on a Monday",
    "Pictures from the Maharashtra election day 2024"
]

count = 0
for query in queries:
    print("-"*50)
    print(f"ORIG QUERY: {query}")
    query2 = replace_methods(query)
    print(f"NEW QUERY: {query2}")
    responses = duck_datetime(query2)
    for response in responses:
        if response['dim']=="time":
            # ==============================
            # HOLIDAYS
            # ==============================
            if "holidayBeta" in response['value']:
                print("HOLIDAY CASE")
                print(f"Date-Time QUERY: {response['body']}")
                # print(f"Spacy's response: {nlp_with_datetime_ner(query)}")
                print(f"isLatent: {response['latent']}")
                print(f"Contents: {response['value']}")

                # CASE 1 : single value specific year
                if response['value']['type']=='value' and len(response['value']['values'])==1 and response['value']['values'][0]['grain']=='day':
                    print(f"match the .date() of {datetime.fromisoformat(response['value']['value'])}")
                
                # CASE 2 : single value, non-specified year (repeats every year)
                # doesn't work on indian festivals given the date varies every year for many 
                if response['value']['type']=='value' and len(response['value']['values'])>1:
                    print(f"Date value: {datetime.fromisoformat(response['value']['value'])}")
                    print(f"Grain level: {response['value']['values'][0]['grain']}")
                    # Calculate delta between two dates
                    date1 = datetime.fromisoformat(response['value']['values'][0]['value'])
                    date2 = datetime.fromisoformat(response['value']['values'][1]['value'])
                    delta = date2 - date1
                    for grain, num in {'week':7, 'month':30, 'year':365}.items():
                        if delta.days//num==1:
                            print(f"Repeat frequency: {grain}")
                    print("Need to check grain from query if it is repetitive or specific day (filter for specific if date value above is in future?)")
                
                # CASE 3 : interval values, specific year
                if response['value']['type']=='interval' and len(response['value']['values'])==1:
                    print(f"Filter date at grain {response['value']['values'][0]['from']['grain']} from {datetime.fromisoformat(response['value']['values'][0]['from']['value'])} to {datetime.fromisoformat(response['value']['values'][0]['to']['value'])}")
                
                # CASE 4 : interval values, non-specified year
                # doesn't work on indian festivals given the date varies every year for many 
                if response['value']['type']=='interval' and len(response['value']['values'])>1:
                    print(f"Filter date at grain {response['value']['values'][0]['from']['grain']} from {datetime.fromisoformat(response['value']['values'][0]['from']['value'])} to {datetime.fromisoformat(response['value']['values'][0]['to']['value'])}")
                    print(f"Filter only day and month if year grain (direct or indirect) is missing in original query")
            
            # ==============================
            # OTHER DAYS
            # ==============================
            else:
                print(f"Date-Time QUERY: {response['body']}")
                # print(f"Spacy's response: {nlp_with_datetime_ner(query)}")
                print(f"isLatent: {response['latent']}")
                print(f"Contents: {response['value']}")

                # CASE 1 : single value specific year
                if response['value']['type']=='value' and len(response['value']['values'])==1 and response['value']['values'][0]['grain']=='day':
                    print(f"match the .date() of {datetime.fromisoformat(response['value']['value'])}")
                elif response['value']['type']=='value' and len(response['value']['values'])==1 and response['value']['values'][0]['grain']=='week':
                    print(f"match the week starting with .date() of {datetime.fromisoformat(response['value']['value'])}")
                elif response['value']['type']=='value' and len(response['value']['values'])==1 and response['value']['values'][0]['grain']=='month':
                    print(f"match the month of .date() of {datetime.fromisoformat(response['value']['value'])}")
                
                # CASE 2 : single value, non-specified year (repeats every year)
                # doesn't work on indian festivals given the date varies every year for many 
                elif response['value']['type']=='value' and len(response['value']['values'])>1:
                    print(f"Date value: {datetime.fromisoformat(response['value']['value'])}")
                    print(f"Grain level: {response['value']['values'][0]['grain']}")
                    # Calculate delta between two dates
                    date1 = datetime.fromisoformat(response['value']['values'][0]['value'])
                    date2 = datetime.fromisoformat(response['value']['values'][1]['value'])
                    delta = date2 - date1
                    for grain, num in {'week':7, 'month':30, 'year':365}.items():
                        if delta.days//num==1:
                            print(f"Repeat frequency: {grain}")
                    print("Need to check grain from query if it is repetitive or specific day (filter for specific if date value above is in future?)")
                
                # CASE 3 : interval values, specific year
                elif response['value']['type']=='interval' and len(response['value']['values'])==1:
                    print(f"Filter date at grain {response['value']['values'][0]['from']['grain']} from {datetime.fromisoformat(response['value']['values'][0]['from']['value'])} to {datetime.fromisoformat(response['value']['values'][0]['to']['value'])}")
                
                # CASE 4 : interval values, non-specified year
                # doesn't work on indian festivals given the date varies every year for many 
                elif response['value']['type']=='interval' and len(response['value']['values'])>1:
                    if datetime.fromisoformat(response['value']['values'][0]['from']['value']) <= local_now:
                        print(f"Filter date at grain {response['value']['values'][0]['from']['grain']} from {datetime.fromisoformat(response['value']['values'][0]['from']['value'])} to {datetime.fromisoformat(response['value']['values'][0]['to']['value'])}")
                #     print(f"Filter only day and month if year grain (direct or indirect) is missing in original query")
    print("-"*50)
    # count+=1
    # if count>2: break


--------------------------------------------------
ORIG QUERY: Images taken during Diwali 2023
NEW QUERY: Images taken during Diwali 2023
HOLIDAY CASE
Date-Time QUERY: Diwali 2023
isLatent: False
Contents: {'grain': 'day', 'holidayBeta': 'Diwali', 'type': 'value', 'value': '2023-11-12T00:00:00.000-08:00', 'values': [{'grain': 'day', 'type': 'value', 'value': '2023-11-12T00:00:00.000-08:00'}]}
match the .date() of 2023-11-12 00:00:00-08:00
--------------------------------------------------
--------------------------------------------------
ORIG QUERY: Photos from Monsoon 2024
NEW QUERY: Photos June to September 2024
Date-Time QUERY: June to September 2024
isLatent: False
Contents: {'from': {'grain': 'month', 'value': '2024-06-01T00:00:00.000-07:00'}, 'to': {'grain': 'month', 'value': '2024-10-01T00:00:00.000-07:00'}, 'type': 'interval', 'values': [{'from': {'grain': 'month', 'value': '2024-06-01T00:00:00.000-07:00'}, 'to': {'grain': 'month', 'value': '2024-10-01T00:00:00.000-07:00'}, 't

In [ ]:
import json

def extract_grains_and_filter_type(duckling_output):
    """
    Extracts grains and determines if the query is an interval or grain-based filter from Duckling output.
    
    Args:
        duckling_output (list): Parsed Duckling output as a list of dictionaries.
    
    Returns:
        tuple: (set of grains, str indicating 'interval' or 'value')
    """
    grains = set()
    filter_type = None
    
    # Filter for time dimension entries
    time_entries = [entry for entry in duckling_output if entry.get('dim') == 'time']
    
    if not time_entries:
        return grains, None  # No time-related entries found
    
    # Process each time entry
    for entry in time_entries:
        value = entry.get('value', {})
        entry_type = value.get('type')
        
        # Determine filter type
        if entry_type == 'interval':
            filter_type = 'interval'
        elif entry_type == 'value':
            filter_type = 'value'
        
        # Extract grains
        if entry_type == 'interval':
            # Check 'from' and 'to' grains
            if 'from' in value:
                from_grain = value['from'].get('grain')
                if from_grain:
                    grains.add(from_grain)
            if 'to' in value:
                to_grain = value['to'].get('grain')
                if to_grain:
                    grains.add(to_grain)
        elif entry_type == 'value':
            # Check single grain
            grain = value.get('grain')
            if grain:
                grains.add(grain)
        
        # Map Duckling grains to required grains (year, month, day, hour, minute)
        if 'year' in grains:
            grains.add('year')
        if 'month' in grains:
            grains.add('month')
            grains.add('year')  # Month implies year
        if 'day' in grains:
            grains.add('day')
            grains.add('month')
            grains.add('year')  # Day implies month and year
        if 'hour' in grains:
            grains.add('hour')
            grains.add('day')
            grains.add('month')
            grains.add('year')  # Hour implies day, month, year
        if 'minute' in grains:
            grains.add('minute')
            grains.add('hour')
            grains.add('day')
            grains.add('month')
            grains.add('year')  # Minute implies hour, day, month, year
    
    return grains, filter_type

# Example usage with the provided sample output
sample_queries = []
count=0
for sample in sample_queries:
    query = sample['query']
    output = sample['output']
    grains, filter_type = extract_grains_and_filter_type(output)
    print(f"QUERY: {sample["query"]}")
    print(f"Grains: {grains}")
    print(f"Filter Type: {filter_type}")
    print("-" * 50)
    count+=1

QUERY: Images taken during Diwali 2023
Grains: {'day', 'year', 'month'}
Filter Type: value
--------------------------------------------------
QUERY: Photos from Monsoon 2024
Grains: {'year', 'month'}
Filter Type: interval
--------------------------------------------------
QUERY: Images taken on a Sunday morning
Grains: {'day', 'year', 'hour', 'month'}
Filter Type: interval
--------------------------------------------------


In [119]:
WEEKDAY_MAP = {
    0: 'Monday',
    1: 'Tuesday',
    2: 'Wednesday',
    3: 'Thursday',
    4: 'Friday',
    5: 'Saturday',
    6: 'Sunday'
}

class DateSearch:
    def __init__(self, find_holidays=True):
        self.find_holidays = find_holidays
        self.datebase = [datetime.fromisoformat(date) for date in np.load(os.path.join('.', 'datetime_metadata.npy'), allow_pickle=True) if date is not None]
        self.grain_order = ["year", "month", "day", "hour", "minute", "second"]
        self.grainrep_year = 2000
        self.grainrep_month = 1
        self.grainrep_day = 1
        self.grainrep_hour = 14

    def __grain_match__(self, dt1, dt2, grain):
        if grain == "year":
            return dt1.year == dt2.year
        elif grain == "month":
            return dt1.year == dt2.year and dt1.month == dt2.month
        elif grain == "day":
            return dt1.year == dt2.year and dt1.month == dt2.month and dt1.day == dt2.day
        elif grain == "hour":
            return (
                dt1.year == dt2.year and
                dt1.month == dt2.month and
                dt1.day == dt2.day and
                dt1.hour == dt2.hour
            )
        elif grain == "minute":
            return (
                dt1.year == dt2.year and
                dt1.month == dt2.month and
                dt1.day == dt2.day and
                dt1.hour == dt2.hour and
                dt1.minute == dt2.minute
            )
        elif grain == "week":
            return ((dt1-dt2).days<7)
        else:
            raise ValueError(f"Unsupported grain: {grain}")

    def __grain_range_match__(self, dt1, dt2, low_grain, high_grain):
        # Define the hierarchy of grains
        
        if high_grain not in self.grain_order or low_grain not in self.grain_order:
            raise ValueError("Invalid grain level. Supported grains: year, month, day, hour, minute, second")

        high_idx = self.grain_order.index(high_grain)
        low_idx = self.grain_order.index(low_grain)

        if low_idx < high_idx - 1:
            grains_to_compare = self.grain_order[low_idx+1:high_idx+1]
        elif low_idx == high_idx - 1:
            grains_to_compare = [high_grain]
        else:
            return False  # Invalid range: low_grain is not below high_grain
        
        # Compare the relevant grains
        for grain in grains_to_compare:
            if getattr(dt1, grain) != getattr(dt2, grain):
                return False

        return True
    
    def __interval_match__(self, dt_in, dt_start, dt_end):
        return (dt_start<=dt_in and dt_in<dt_end)
    
    def __replace_grain__(self, dt, grain):
        self.grainrep_year
        if grain=='year':
            return dt.replace(year=self.grainrep_year)
        elif grain=='month':
            return dt.replace(year=self.grainrep_year, month=self.grainrep_month)
        elif grain=='day':
            return dt.replace(year=self.grainrep_year, month=self.grainrep_month, day=self.grainrep_day)
        elif grain=='hour':
            return dt.replace(year=self.grainrep_year, month=self.grainrep_month, day=self.grainrep_day, hour=self.grainrep_hour)

    
    def search_point_in_time(self, dt_in, grain):
        count=0
        for i, dt in enumerate(self.datebase):
            if self.__grain_match__(dt_in, dt, grain):
                if count<10:
                    print(f"Matched date: {dt}, index: {i}")
                count+=1
        print(f"Total Matches Found: {count}")
        pass
    
    def day_of_week_lookup(self, dt_in, high_grain):
        count=0
        for i, dt in enumerate(self.datebase):
            if dt.weekday()==dt_in.weekday() and (self.__grain_range_match__(dt_in, dt, 'day', high_grain) if high_grain in ['hour', 'minute', 'second'] else True):
                if count<10:
                    # print(WEEKDAY_MAP[dt.weekday()])
                    # print(WEEKDAY_MAP[dow])
                    print(f"Matched date: {dt}, index: {i}")
                count+=1
        print(f"Total Matches Found: {count}")
        pass
    
    def repeat_lookup(self, dt_in, low_grain, high_grain):
        count=0
        for i, dt in enumerate(self.datebase):
            if self.__grain_range_match__(dt_in, dt, low_grain, high_grain):
                if count<10:
                    print(f"Matched date: {dt}, index: {i}")
                count+=1
        print(f"Total Matches Found: {count}")
    
    def interval_lookup(self, dt_start=datetime.fromisoformat('1980-01-01T00:00:00+05:30'), dt_end=datetime.fromisoformat("2050-12-31T11:59:59+05:30")):
        count=0
        for i, dt in enumerate(self.datebase):
            if self.__interval_match__(dt, dt_start, dt_end):
                if count<10:
                    print(f"Matched date: {dt}, index: {i}")
                count+=1
        print(f"Total Matches Found: {count}")

    def repeat_interval_lookup(self, dt_start, dt_end, rept_freq):
        count=0
        print(f"Start: {dt_start}, End: {dt_end}")
        
        start_dt = self.__replace_grain__(dt_start, rept_freq)
        end_dt = self.__replace_grain__(dt_end, rept_freq)
        print(f"Start: {start_dt}, End: {end_dt}")
        for i, dt in enumerate(self.datebase):
            dt2 = self.__replace_grain__(dt, rept_freq)
            # print(f"Date compared: {dt2}")
            if start_dt>end_dt:
                if start_dt<=dt2 or dt2<end_dt:
                    if count<10:
                        print(f"Matched date: {dt}, index: {i}")
                    count+=1
            else:
                if start_dt<=dt2 and dt2<end_dt:
                    if count<10:
                        print(f"Matched date: {dt}, index: {i}")
                    count+=1

            
        print(f"Total Matches Found: {count}")
            
        
        


In [120]:
dt_data[:5]

['2023-03-18T19:44:56+05:30',
 '2016-07-20T15:27:22+05:30',
 '2022-10-19T08:41:55+05:30',
 '2021-03-30T09:14:05+05:30',
 '2021-03-30T12:12:50+05:30']

In [ ]:
queries = [
    "Images taken during Diwali 2023",
    "Pictures captured on Republic day 2023",
    "Photos from Monsoon 2024",
    "Photos taken between October 15, 2024, and November 15, 2024",
    "Show me photos from Holi 2022",
    "Images from last year's Durga Puja",
    "Pictures taken on Ugadi 2023",
    "Show me photos from last Christmas",
    "Find images from September 2023",
    "Pictures from the summer of 2020",
    "Show me photos from December 2023 in Mumbai",
    "Photos captured on Independence Day 2024",
    "Show me images taken around sunset last Tuesday of 2024",
    "Photos from Ganesh Chaturthi 2020",
    "August vibes",
    "Photos from the past year in Delhi",
    "Vietnam trip from march 2024",
    "Pictures from this morning in Bengaluru",
    "Images from a random Sunday",
    "Photos from around 6 AM on a Monday",
    "Pictures taken during afternoon on december 12, 2024",
    "Dhruv before 2020",
    "photos after jun 23rd 2023",
    "Images of garba during navratri",
    "Show me photos from the winter of 2022",
    "jets flying in sky between 10am and 12pm", # also includes time till 1pm for some reason
    "Find images from the New Year's Eve party 2023", # doesn't recognize 2023
    "Images take on a november sunday", # doesn't filter november
    "photos between 9am and 12pm in july", # doesn't filter july, also includes filter till 1pm for some reason
    "Images captured before sunrise on a Sunday in Dubai", # "before" case, only captures "on a sunday"
    "Images from March to May 2023 ", # only captures "March"
    "Images taken on a Sunday morning", # haven't coded this case
    "Images captured after 7 PM in Chennai", # "after" case 
]


dt_search = DateSearch(find_holidays=True)
count = 0
for query in queries:
    print("-"*50)
    print(f"ORIG QUERY: {query}")
    query2 = replace_methods(query)
    print(f"NEW QUERY: {query2}")
    responses = duck_datetime(query2)
    for response in responses:
        # print(json.dumps(response, indent=4))
        if response['dim']=="time":
            print(f"Date-Time QUERY: {response['body']}")
            # print(f"Spacy's response: {nlp_with_datetime_ner(query)}")
            print(f"Contents: {response['value']}")

            # CASE 1 : single value specific year up to any grain level
            if response['value']['type']=='value' and len(response['value']['values'])==1:
                print("CASE 1")
                print(f"Match the .date() at {response['value']['values'][0]['grain']} grain of {datetime.fromisoformat(response['value']['value'])}")
                dt_search.search_point_in_time(dt_in=datetime.fromisoformat(response['value']['value']), grain=response['value']['values'][0]['grain'])
            
            # CASE 2 : multi-value specific year up to any grain level
            if response['value']['type']=='value' and len(response['value']['values'])>1 and datetime.fromisoformat(response['value']['values'][-1]['value']) <= local_now:
                print("CASE 2")
                print(f"Match the .date() at {response['value']['values'][0]['grain']} grain of {datetime.fromisoformat(response['value']['value'])}")
                # Calculate delta between two dates
                dt_search.search_point_in_time(dt_in=datetime.fromisoformat(response['value']['values'][0]['value']), grain=response['value']['values'][0]['grain'])

            # CASE 3 : multi-value repetitive dates with granularity range
            if response['value']['type']=='value' and len(response['value']['values'])>1 and datetime.fromisoformat(response['value']['values'][-1]['value']) > local_now:
                print("CASE 3")
                print(f"Match the .date() at {response['value']['values'][0]['grain']} grain of {datetime.fromisoformat(response['value']['value'])}")
                # Calculate delta between two dates
                date1 = datetime.fromisoformat(response['value']['values'][0]['value'])
                date2 = datetime.fromisoformat(response['value']['values'][1]['value'])
                delta = date2 - date1
                for rept_freq, num in {'week':7, 'month':30, 'year':365}.items():
                    if delta.days//num==1:
                        print(f"Repeat frequency: {rept_freq}")
                        if rept_freq=='week':
                            dt_search.day_of_week_lookup(dt_in=date1, high_grain=response['value']['values'][0]['grain'])
                        if rept_freq=='year' or rept_freq=='month':
                            dt_search.repeat_lookup(date1, high_grain=response['value']['values'][0]['grain'], low_grain=rept_freq)
            
            # CASE 4 : single value fixed interval
            if response['value']['type']=='interval' and len(response['value']['values'])==1:
                print("CASE 4")
                print(f"Restrict the .date() at {response['value']['values'][0]['from']['grain'] if 'from' in response['value']['values'][0].keys() else response['value']['values'][0]['to']['grain']} grain between {datetime.fromisoformat(response['value']['values'][0]['from']['value']) if 'from' in response['value']['values'][0].keys() else "???"} and {datetime.fromisoformat(response['value']['values'][0]['to']['value']) if 'to' in response['value']['values'][0].keys() else "???"}")
                if 'from' in response['value']['values'][0].keys() and 'to' in response['value']['values'][0].keys():
                    dt_search.interval_lookup(dt_start=datetime.fromisoformat(response['value']['values'][0]['from']['value']), dt_end=datetime.fromisoformat(response['value']['values'][0]['to']['value']))
                elif 'from' in response['value']['values'][0].keys():
                    dt_search.interval_lookup(dt_start=datetime.fromisoformat(response['value']['values'][0]['from']['value']))
                elif 'to' in response['value']['values'][0].keys():
                    dt_search.interval_lookup(dt_end=datetime.fromisoformat(response['value']['values'][0]['to']['value']))
            
            # CASE 5 : multi-value fixed interval
            if response['value']['type']=='interval' and len(response['value']['values'])>1 and (datetime.fromisoformat(response['value']['values'][-1]['from']['value']) if 'from' in response['value']['values'][-1].keys() else datetime.fromisoformat(response['value']['values'][-1]['to']['value'])) <= local_now:
                print("CASE 5")
                print(f"Restrict the .date() at {response['value']['values'][0]['from']['grain'] if 'from' in response['value']['values'][0].keys() else response['value']['values'][0]['to']['grain']} grain between {datetime.fromisoformat(response['value']['values'][0]['from']['value']) if 'from' in response['value']['values'][0].keys() else "???"} and {datetime.fromisoformat(response['value']['values'][0]['to']['value']) if 'to' in response['value']['values'][0].keys() else "???"}")
                dt_search.interval_lookup(dt_start=datetime.fromisoformat(response['value']['values'][0]['from']['value']), dt_end=datetime.fromisoformat(response['value']['values'][0]['to']['value']))
            
            # CASE 6 : multi-value sliding interval
            if response['value']['type']=='interval' and len(response['value']['values'])>1 and (datetime.fromisoformat(response['value']['values'][-1]['from']['value']) if 'from' in response['value']['values'][-1].keys() else datetime.fromisoformat(response['value']['values'][-1]['to']['value'])) > local_now:
                print("CASE 6")
                print(f"Restrict the .date() at {response['value']['values'][0]['from']['grain'] if 'from' in response['value']['values'][0].keys() else response['value']['values'][0]['to']['grain']} grain between {datetime.fromisoformat(response['value']['values'][0]['from']['value']) if 'from' in response['value']['values'][0].keys() else "???"} and {datetime.fromisoformat(response['value']['values'][0]['to']['value']) if 'to' in response['value']['values'][0].keys() else "???"}")
                # print(f"Restrict the .date() at {response['value']['values'][0]['from']['grain']} grain between {datetime.fromisoformat(response['value']['values'][0]['from']['value'])} and {datetime.fromisoformat(response['value']['values'][0]['to']['value'])}")
                # dt_search.interval_lookup(dt_start=datetime.fromisoformat(response['value']['values'][0]['from']['value']), dt_end=datetime.fromisoformat(response['value']['values'][0]['to']['value']))
                # Calculate delta between two dates
                date1from = datetime.fromisoformat(response['value']['values'][0]['from']['value'] if 'from' in response['value']['values'][0].keys() else response['value']['values'][0]['to']['value'])
                date2 = datetime.fromisoformat(response['value']['values'][1]['from']['value'] if 'from' in response['value']['values'][1].keys() else response['value']['values'][1]['to']['value'])
                date1to = datetime.fromisoformat(response['value']['values'][0]['to']['value'] if 'to' in response['value']['values'][0].keys() else response['value']['values'][0]['from']['value'])
                delta = date2 - date1from
                for rept_freq, num in {'day':1, 'week':7, 'month':30, 'year':365}.items():
                    if delta.days//num==1:
                        print(f"Repeat frequency: {rept_freq}")
                        if rept_freq in ['year', 'month', 'day']:
                            dt_search.repeat_interval_lookup(date1from, date1to, rept_freq)

